# GTHA housing market database
# OSEMN methodology Step 1: Obtain
# Obtain Select DA-level Census Variables

---

This notebook describes _Step 1: Obtain_ of OSEMN methodology, the process of select DA-level Census variables.

---

For description of OSEMN methodology, see `methodology/0.osemn/osemn.pdf`.

## Import dependencies

In [1]:
import pandas as pd
import os
from time import time

In [4]:
data_path = '../../../data/census/taz_level_vars/'
os.listdir(data_path)

['taz_census_age_edu_employment.xlsx', 'taz_census_housing_monetaryvars.xlsx']

## Load geometry of GTHA Dissemination Areas (DAs)

In [5]:
t = time()
df = pd.read_excel(data_path + 'taz_census_age_edu_employment.xlsx.xls')
elapsed = time() - t
print("\n----- DataFrame loaded"
      "\nin {0:.2f} seconds".format(elapsed) + 
      "\nwith {0:,} rows\nand {1:,} columns"
      .format(df.shape[0], df.shape[1]) + 
      "\n-- Column names:\n", df.columns)

ImportError: Missing optional dependency 'xlrd'. Install xlrd >= 1.0.0 for Excel support Use pip or conda to install xlrd.

## Convert data to tidy format
Same variables are stored in several columns corresponding to different Census years, which violates one of the conditions of Tidy Data. To correct this, a new column `year` will be added to be used as a part of a **_primary key_** along with `dauid` for the `da_census_select` table in the database. DataFrame will be melted for each variable to be stored in a single column additionally referenced by the column `year`.

### Desired shape of the DataFrame
There are 30 unique variables for each Census year, 31 unique variables in total (variable `Self-emp` gets renamed to `Self_emp` in 2016 Census, to be corrected after melting).

In [5]:
mask1 = df.columns.str.contains('01')
cols = df.columns[mask1]
len(cols)

30

#### All Census variables in the original table

In [6]:
var_names = df.columns[df.columns.str.contains('\d')]
var_names = var_names.str.slice(stop=-2)
var_names = var_names.unique()
len(var_names)

31

In [7]:
var_names

Index(['Avg_HHsize', 'Avg_HHinc', 'Avg_own_payt', 'Avg_val_dwel', 'Avg_rent',
       'Pop', 'PopDens', 'Dwel', 'DwelDens', 'Sgl_det', 'Apt_5plus', 'Sgl_att',
       'Owned', 'Rented', 'CarTrVan_d', 'CarTrVan_p', 'PT', 'Walk', 'Bike',
       'Lbrfrc', 'Emp', 'Unemp', 'Not_lbrfrc', 'Employee', 'Self_emp',
       'At_home', 'No_fix_wkpl', 'Usl_wkpl', 'Blue_cljob', 'White_cljob',
       'Self-emp'],
      dtype='object')

There are four different Census years:

In [8]:
mask1 = df.columns.str.contains('Avg_HHsize')
cols = df.columns[mask1]
cols

Index(['Avg_HHsize16', 'Avg_HHsize11', 'Avg_HHsize06', 'Avg_HHsize01'], dtype='object')

There are 9'182 Dissemination Areas.

In [9]:
df.index.nunique()

9182

In [10]:
len(df)

9182

In [11]:
len(df) * 4

36728

That means, that after melting we should get a table with 9'182 x 4 = 36'728 rows and 32 columns (`DAUID`, `year`, and 31 columns with Census variables).

### Melt all the variables in the table

In [14]:
i = 0

for var_name in var_names:

    # select a subset of columns containing different years of a Census variable
    mask1 = df.columns.str.contains('{0}\d'.format(var_name))
    var_cols = df.columns[mask1]
    s = df[['DAUID'] + list(var_cols)]

    df_melt = pd.melt(s, id_vars='DAUID', value_name=var_name)
    df_melt['variable'] = '20' + df_melt['variable'].str.slice(-2)
    df_melt = df_melt.rename(columns={'variable': 'year'})
    df_melt['year'] = df_melt['year'].astype('int')
    if i  == 0:
        df_tidy = df_melt.sort_values('DAUID')
    else:
        df_tidy = pd.merge(df_tidy, df_melt, how='left', 
                           left_on=['DAUID', 'year'], right_on=['DAUID', 'year'])
    i += 1

df_tidy.sort_values(['DAUID', 'year'])

,DAUID,year,Avg_HHsize,Avg_HHinc,Avg_own_payt,Avg_val_dwel,Avg_rent,Pop,PopDens,Dwel,...,Unemp,Not_lbrfrc,Employee,Self_emp,At_home,No_fix_wkpl,Usl_wkpl,Blue_cljob,White_cljob,Self-emp
3,35180017,2001,2.945455,82963.090909,1079.181818,248639.000000,492.363636,883,14.267301,284.0,...,10.0,152.0,390.0,NaN,155.0,30.0,335.0,236.0,295.0,141.0
2,35180017,2006,2.958333,105432.416667,1336.583333,397211.500000,672.250000,843,13.620991,281.0,...,0.0,168.0,422.0,NaN,105.0,67.0,377.0,172.0,357.0,121.0
1,35180017,2011,2.841667,124624.250000,1565.750000,537326.583333,336.250000,871,14.073408,292.0,...,1.0,154.0,435.0,NaN,56.0,76.0,380.0,258.0,260.0,121.0
0,35180017,2016,2.900000,137158.000000,1764.000000,673145.000000,1224.000000,845,13.653306,290.0,...,20.0,170.0,440.0,130.0,100.0,80.0,380.0,225.0,350.0,NaN
6,35180018,2001,2.837500,72401.875000,1101.625000,230609.500000,654.875000,681,111.155307,241.0,...,25.0,103.0,330.0,NaN,39.0,20.0,310.0,103.0,310.0,74.0
5,35180018,2006,2.900000,85806.285714,1407.142857,370896.142857,740.857143,1265,206.477920,419.0,...,25.0,207.0,576.0,NaN,39.0,138.0,488.0,167.0,502.0,113.0
4,35180018,2011,2.800000,112465.714286,1544.857143,464213.571429,596.428571,1540,251.364424,507.0,...,44.0,251.0,793.0,NaN,84.0,103.0,680.0,163.0,685.0,103.0
7,35180018,2016,3.100000,131452.000000,1842.000000,592286.000000,1378.000000,1573,256.750805,510.0,...,90.0,295.0,850.0,130.0,135.0,90.0,685.0,235.0,765.0,NaN
9,35180019,2001,2.575000,67275.500000,1067.500000,212928.500000,611.500000,741,2828.825785,261.0,...,10.0,165.0,286.0,NaN,55.0,30.0,281.0,75.0,286.0,75.0
10,35180019,2006,2.675000,79505.500000,1393.250000,334109.000000,959.500000,728,2779.197263,271.0,...,15.0,201.0,322.0,NaN,30.0,50.0,281.0,120.0,266.0,55.0


#### Assign values of `Self_emp` to `Self-emp`

In [15]:
mask1 = df_tidy['year'] == 2016
df_tidy.loc[mask1, 'Self_emp'].isnull().sum()

35

In [16]:
df_tidy.loc[~mask1, 'Self_emp'].isnull().sum()

27546

In [17]:
df_tidy.loc[~mask1, 'Self_emp'] = df_tidy.loc[~mask1, 'Self-emp']
df_tidy.loc[~mask1, 'Self_emp'].isnull().sum()

30

#### Drop column `Self-emp`

In [18]:
df_tidy = df_tidy.drop('Self-emp', axis=1)
df_tidy.columns

Index(['DAUID', 'year', 'Avg_HHsize', 'Avg_HHinc', 'Avg_own_payt',
       'Avg_val_dwel', 'Avg_rent', 'Pop', 'PopDens', 'Dwel', 'DwelDens',
       'Sgl_det', 'Apt_5plus', 'Sgl_att', 'Owned', 'Rented', 'CarTrVan_d',
       'CarTrVan_p', 'PT', 'Walk', 'Bike', 'Lbrfrc', 'Emp', 'Unemp',
       'Not_lbrfrc', 'Employee', 'Self_emp', 'At_home', 'No_fix_wkpl',
       'Usl_wkpl', 'Blue_cljob', 'White_cljob'],
      dtype='object')

#### Validate the shape of the melted DataFrame

In [19]:
df.shape

(9182, 124)

In [20]:
df_tidy.shape

(36728, 32)

## Save results to a .csv file

In [188]:
save_path = data_path + 'da_census_select_tidy.csv'
t = time()
df_tidy.sort_values(['DAUID', 'year']).to_csv(save_path, index=False)
elapsed = time() - t
print("DataFrame saved to file:\n", save_path,
      "\ntook {0:.2f} seconds".format(elapsed))

DataFrame saved to file:
 ../../../data/da_census/da_census_select_tidy.csv 
took 1.81 seconds
